In [1]:
import scipy as scipy
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import table
import math

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/saharsh/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [10]:
hdu_lst = fits.open('glg_tte_n4_bn200829582_v00.fit')

In [11]:
hdu_lst[2].header

XTENSION= 'BINTABLE'           / marks beginning of new HDU                     
BITPIX  =                    8 / bits per data value                            
NAXIS   =                    2 / number of axes                                 
NAXIS1  =                   10 / size of the n'th axis                          
NAXIS2  =               851526 / size of the n'th axis                          
PCOUNT  =                    0 / Required value                                 
GCOUNT  =                    1 / Required value                                 
TFIELDS =                    2 / Number of table fields                         
CHECKSUM= 'IlAnLkAmIkAmIkAm'   / checksum for the current HDU                   
DATASUM = '2386174270'         / checksum of the data records                   
TFORM1  = '1D      '           / column data format                             
TTYPE1  = 'TIME    '           / label for field   1                            
TUNIT1  = 's       '        

In [20]:
def single_peak_test(a, t90, t90_start, trig_time):
    hdu_list = fits.open(a)

    tstop = hdu_list[3].data['STOP'][0]
    tstart = hdu_list[3].data['START'][0]

    filter_conditions = (hdu_list[2].data['PHA'] > 33) & (hdu_list[2].data['PHA'] <= 124)
    filtered_data = hdu_list[2].data[filter_conditions]

    t90_region_filter = (filtered_data['TIME'] >= trig_time + t90_start) & (filtered_data['TIME'] <= trig_time + t90_start + t90)
    t90_data = filtered_data[t90_region_filter]

    bg_region_filter = (filtered_data['TIME'] < trig_time + t90_start) | (filtered_data['TIME'] > trig_time + t90_start + t90)
    bg_data = filtered_data[bg_region_filter]

    hist_t90, _ = np.histogram(t90_data['TIME'].astype('float128'), bins=120)
    hist_norm_factor = t90/120
    hist_t90 = hist_t90/hist_norm_factor
    hist_bg, _ = np.histogram(bg_data['TIME'].astype('float128'), bins=120)
    #bg_norm_factor = ((trig_time +  - tstart) + (tstop - t90 - t90_start-trig_time)) / 120
    bg_norm_factor =  (tstop - tstart - t90)/120
    hist_bg = hist_bg / bg_norm_factor

    bg = np.median(hist_bg)
    peak = hist_t90.max()
    print(peak,bg)

    tline = peak - 2 * math.sqrt(peak)
    bline = bg + 2 * math.sqrt(bg)
    lines = np.linspace(bline, tline, 16)[1:15]

    test_score = 0

    for line in lines:
        err = np.sqrt(line)
        above_points = np.where(hist_t90 >= line + err)[0]

        if len(above_points) < 2:
            continue

        first, last = above_points[0], above_points[-1]

        below_points = hist_t90[first:last + 1] < (line - err)
        test_score += np.sum(below_points)

    n = t90 / 120
    normalized_score = 1 - (test_score / (120 * 15))

    print("Normalized single-peak score:", normalized_score)
    if normalized_score>=0.9983:
        print('Single peaked GRB')
    else:
        print('Multiple peaked GRB')
    hdu_list.close()

In [21]:
trig_tm = 620402299.660388
start = 16.896
duration = 6.912
single_peak_test('glg_tte_n4_bn200829582_v00.fit',duration,start,trig_tm)

10347.222222222223 2031.0
Normalized single-peak score: 0.9727777777777777
Multiple peaked GRB
